<a href="https://colab.research.google.com/github/ravi-gopalan/singapore_hdb_price_analysis/blob/master/singapore_hdb_resale_transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview of the dataset and the purpose

Singapore is a city state and Property ownership is a hallmark of the economy. Land is scarce and the Government encourages all residents to work towards home ownership.

Singapore residents stay in either of private condominia or Housing Development Board (HDB) apartments or bungalows. The largest chunk of the population stay in HDB apartments either as owners or renters. They purchase apartments either when they are newly constructed or resold. Resale of HDB apartments is a strong barometer of the broader Singapore economy. The data of HDB resale transactions have been made available through the Singapore Open Data project https://data.gov.sg/.
Data for HDB resales are available since 1990 organized by each individual transaction including month of transaction, broad location of the property (town and street) as well as type of apartment. The following link has been used to obtain the raw data for analysis https://data.gov.sg/dataset/resale-flat-prices

Data is provided at https://data.gov.sg/dataset/7a339d20-3c57-4b11-a695-9348adfd7614/download
and has 4 csv files zipped together

In [0]:
import requests
import pandas as pd
import re
import time
from geopy.geocoders import Nominatim


In [2]:
# Clone the entire repo.
!git clone -l -s git://github.com/ravi-gopalan/singapore_hdb_price_analysis.git cloned-repo
!cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 255.61 KiB | 2.48 MiB/s, done.
cloned-repo  sample_data


In [0]:
url = 'https://data.gov.sg/dataset/7a339d20-3c57-4b11-a695-9348adfd7614/download'
file_name = 'resale_flat_prices.zip'
req = requests.get(url)
file = open(file_name, 'wb')
for chunk in req.iter_content(100000):
    file.write(chunk)
file.close()

Data is provided at https://data.gov.sg/dataset/7a339d20-3c57-4b11-a695-9348adfd7614/download and has 5 csv files zipped ogether

In [4]:
!unzip resale_flat_prices.zip

Archive:  resale_flat_prices.zip
 extracting: metadata-resale-flat-prices.txt  
 extracting: resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv  
 extracting: resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv  
 extracting: resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv  
 extracting: resale-flat-prices-based-on-approval-date-2000-feb-2012.csv  
 extracting: resale-flat-prices-based-on-approval-date-1990-1999.csv  


In [0]:
df1 = pd.read_csv('resale-flat-prices-based-on-approval-date-1990-1999.csv')
df2 = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
df3 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
df4 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
df5 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

In [6]:
df = pd.concat([df1, df2, df3, df4, df5],axis=0)
df.reset_index(inplace=True)
df.drop(columns=['index'],inplace=True)
df.head()
df.tail()
df.info()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804361 entries, 0 to 804360
Data columns (total 11 columns):
block                  804361 non-null object
flat_model             804361 non-null object
flat_type              804361 non-null object
floor_area_sqm         804361 non-null float64
lease_commence_date    804361 non-null int64
month                  804361 non-null object
remaining_lease        95307 non-null object
resale_price           804361 non-null float64
storey_range           804361 non-null object
street_name            804361 non-null object
town                   804361 non-null object
dtypes: float64(2), int64(1), object(8)
memory usage: 67.5+ MB


In [7]:
df['resale_month'] = pd.to_datetime(df['month'],format='%Y-%m')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804361 entries, 0 to 804360
Data columns (total 12 columns):
block                  804361 non-null object
flat_model             804361 non-null object
flat_type              804361 non-null object
floor_area_sqm         804361 non-null float64
lease_commence_date    804361 non-null int64
month                  804361 non-null object
remaining_lease        95307 non-null object
resale_price           804361 non-null float64
storey_range           804361 non-null object
street_name            804361 non-null object
town                   804361 non-null object
resale_month           804361 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(8)
memory usage: 73.6+ MB


In [8]:
df['flat_model'] = df['flat_model'].str.title()
df.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town,resale_month
0,309,Improved,1 ROOM,31.0,1977,1990-01,NaN,9000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01
1,309,Improved,1 ROOM,31.0,1977,1990-01,NaN,6000.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01
2,309,Improved,1 ROOM,31.0,1977,1990-01,NaN,8000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01
3,309,Improved,1 ROOM,31.0,1977,1990-01,NaN,6000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01
4,216,New Generation,3 ROOM,73.0,1976,1990-01,NaN,47200.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01


In [9]:
df['flat_type'] = df['flat_type'].str.title()
df['flat_type'].value_counts()
flat_type = {'1 Room': '1R', '2 Room': '2R', '3 Room': '3R', \
             '4 Room': '4R', '5 Room': '5R', 'Executive': 'exec', \
             'Multi Generation': 'multi', 'Multi-Generation': 'multi'}
df['flat_type'] = df['flat_type'].map(flat_type)
df['flat_type'].value_counts()

4R       300061
3R       267262
5R       164900
exec      60954
2R         9423
1R         1265
multi       496
Name: flat_type, dtype: int64

In [10]:
df.isna().sum()

block                       0
flat_model                  0
flat_type                   0
floor_area_sqm              0
lease_commence_date         0
month                       0
remaining_lease        709054
resale_price                0
storey_range                0
street_name                 0
town                        0
resale_month                0
dtype: int64

In [11]:
df['resale_year'] = df['resale_month'].dt.year
df.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town,resale_month,resale_year
0,309,Improved,1R,31.0,1977,1990-01,NaN,9000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990
1,309,Improved,1R,31.0,1977,1990-01,NaN,6000.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990
2,309,Improved,1R,31.0,1977,1990-01,NaN,8000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990
3,309,Improved,1R,31.0,1977,1990-01,NaN,6000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990
4,216,New Generation,3R,73.0,1976,1990-01,NaN,47200.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990


In [12]:
df.tail()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town,resale_month,resale_year
804356,326,Maisonette,exec,146.0,1988,2019-09,67 years 10 months,543888.0,04 TO 06,YISHUN RING RD,YISHUN,2019-09-01,2019
804357,640A,Apartment,exec,164.0,1992,2019-09,71 years 09 months,695888.0,01 TO 03,YISHUN ST 61,YISHUN,2019-09-01,2019
804358,636A,Apartment,exec,164.0,1992,2019-09,71 years 09 months,745000.0,01 TO 03,YISHUN ST 61,YISHUN,2019-09-01,2019
804359,723,Maisonette,exec,146.0,1986,2019-09,65 years 08 months,605000.0,10 TO 12,YISHUN ST 71,YISHUN,2019-09-01,2019
804360,836,Maisonette,exec,146.0,1988,2019-09,67 years 05 months,640000.0,04 TO 06,YISHUN ST 81,YISHUN,2019-09-01,2019


The data frame has 804,361 observations along 11 variables that cover the apartment sold (i.e. block, street_name, town, storey_range, flat_model, flat_type, floor area in square meters). In addition the data includes the month of the transaction, resale_price as well as year lease commenced and how many years remain on the lease.

It also indicates that apart from the lease commencement and years remaining as integers, and the floor area as numbers all other variables are in character format.

Floor areas for 434 observations (~0.06%) are missing. One may need to analyze the data and see if these could be imputed.
Remaining lease data is missing for ~710k of the observations (~90%). 

We could look to define some of the fields as categories, manipulate column names, standardize some names and also enrich if possible.

The data frame is then manipulated for column names, factors, time fields, enriched values

In [13]:
df['storey_range'].value_counts()

04 TO 06    204464
07 TO 09    184551
01 TO 03    164845
10 TO 12    156218
13 TO 15     50070
16 TO 18     18435
19 TO 21      9019
22 TO 24      5766
01 TO 05      2700
06 TO 10      2474
25 TO 27      2361
11 TO 15      1259
28 TO 30       958
16 TO 20       265
34 TO 36       237
37 TO 39       225
31 TO 33       224
40 TO 42       112
21 TO 25        92
26 TO 30        39
46 TO 48        17
43 TO 45        13
49 TO 51         8
36 TO 40         7
31 TO 35         2
Name: storey_range, dtype: int64

In [14]:
regex_storeys = re.compile(r' TO ', flags=re.IGNORECASE)
df['storey_range'] = df['storey_range'].str.replace(regex_storeys,'-')
df['storey_range'].value_counts()

04-06    204464
07-09    184551
01-03    164845
10-12    156218
13-15     50070
16-18     18435
19-21      9019
22-24      5766
01-05      2700
06-10      2474
25-27      2361
11-15      1259
28-30       958
16-20       265
34-36       237
37-39       225
31-33       224
40-42       112
21-25        92
26-30        39
46-48        17
43-45        13
49-51         8
36-40         7
31-35         2
Name: storey_range, dtype: int64

In [0]:
df_remaining_lease = df['remaining_lease']

In [16]:
df_remaining_lease = df_remaining_lease.str.split(' ',expand=True)
df_remaining_lease.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804361 entries, 0 to 804360
Data columns (total 4 columns):
0    58154 non-null object
1    58154 non-null object
2    53527 non-null object
3    53527 non-null object
dtypes: object(4)
memory usage: 24.5+ MB


In [17]:
df_remaining_lease.tail(10)

,0,1,2,3
804351,67,years,09,months
804352,67,years,09,months
804353,68,years,02,months
804354,67,years,10,months
804355,68,years,02,months
804356,67,years,10,months
804357,71,years,09,months
804358,71,years,09,months
804359,65,years,08,months
804360,67,years,05,months


In [18]:
df_remaining_lease.columns = ['first','second','third','fourth']
df_remaining_lease.head()
df_remaining_lease.head()

,first,second,third,fourth
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
df_remaining_lease.drop(columns=['second','fourth'],inplace=True)
df_remaining_lease['first'].fillna(100000,inplace=True)
df_remaining_lease['third'].fillna(0,inplace=True)
df_remaining_lease = df_remaining_lease.astype({'first': 'int64','third': 'float64' },copy=True)
df_remaining_lease['lease_term_remaining'] = df_remaining_lease['first'] + (df_remaining_lease['third']/12)
df_remaining_lease.head()
df_remaining_lease.info()

,first,third,lease_term_remaining
0,100000,0.0,100000.0
1,100000,0.0,100000.0
2,100000,0.0,100000.0
3,100000,0.0,100000.0
4,100000,0.0,100000.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804361 entries, 0 to 804360
Data columns (total 3 columns):
first                   804361 non-null int64
third                   804361 non-null float64
lease_term_remaining    804361 non-null float64
dtypes: float64(2), int64(1)
memory usage: 18.4 MB


In [0]:
df_remaining_lease['lease_term_remaining'] = df_remaining_lease['first'] + (df_remaining_lease['third']/12)

In [0]:
df_remaining_lease['lease_term_remaining'].replace(to_replace= 100000.0,value=0,inplace=True)

In [0]:
df_remaining_lease.drop(columns=['first','third'],inplace=True)

In [24]:
df_remaining_lease.head()
df_remaining_lease.tail()

,lease_term_remaining
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


,lease_term_remaining
804356,67.833333
804357,71.750000
804358,71.750000
804359,65.666667
804360,67.416667


In [25]:
df = pd.concat([df,df_remaining_lease],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804361 entries, 0 to 804360
Data columns (total 14 columns):
block                   804361 non-null object
flat_model              804361 non-null object
flat_type               804361 non-null object
floor_area_sqm          804361 non-null float64
lease_commence_date     804361 non-null int64
month                   804361 non-null object
remaining_lease         95307 non-null object
resale_price            804361 non-null float64
storey_range            804361 non-null object
street_name             804361 non-null object
town                    804361 non-null object
resale_month            804361 non-null datetime64[ns]
resale_year             804361 non-null int64
lease_term_remaining    804361 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(2), object(8)
memory usage: 85.9+ MB


In [26]:
df.drop(columns=['remaining_lease'],inplace=True)
df.tail()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,resale_price,storey_range,street_name,town,resale_month,resale_year,lease_term_remaining
804356,326,Maisonette,exec,146.0,1988,2019-09,543888.0,04-06,YISHUN RING RD,YISHUN,2019-09-01,2019,67.833333
804357,640A,Apartment,exec,164.0,1992,2019-09,695888.0,01-03,YISHUN ST 61,YISHUN,2019-09-01,2019,71.750000
804358,636A,Apartment,exec,164.0,1992,2019-09,745000.0,01-03,YISHUN ST 61,YISHUN,2019-09-01,2019,71.750000
804359,723,Maisonette,exec,146.0,1986,2019-09,605000.0,10-12,YISHUN ST 71,YISHUN,2019-09-01,2019,65.666667
804360,836,Maisonette,exec,146.0,1988,2019-09,640000.0,04-06,YISHUN ST 81,YISHUN,2019-09-01,2019,67.416667


In [0]:
df['street'] = df['street_name']

In [64]:
street_names = pd.DataFrame(df['street_name'].unique())
type(street_names)

pandas.core.frame.DataFrame

In [65]:
street_names.columns=['street_name']
street_names.info()
street_names.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 1 columns):
street_name    559 non-null object
dtypes: object(1)
memory usage: 4.4+ KB


,street_name
0,ANG MO KIO AVE 1
1,ANG MO KIO AVE 3
2,ANG MO KIO AVE 4
3,ANG MO KIO AVE 10
4,ANG MO KIO AVE 5


In [66]:
street_names['updated_street'] = street_names['street_name']+ ';'
street_names.tail()

,street_name,updated_street
554,YISHUN AVE 1,YISHUN AVE 1;
555,ANCHORVALE CRES,ANCHORVALE CRES;
556,ANCHORVALE ST,ANCHORVALE ST;
557,TAMPINES CTRL 8,TAMPINES CTRL 8;
558,YISHUN ST 51,YISHUN ST 51;


In [0]:
regex_street_1 = re.compile(r' ST ', flags=re.IGNORECASE)
regex_street_2 = re.compile(r' ST;', flags=re.IGNORECASE)

regex_road_1 = re.compile(r' RD ', flags=re.IGNORECASE)
regex_road_2 = re.compile(r' RD;', flags=re.IGNORECASE)

regex_lane_1 = re.compile(r' LN ', flags=re.IGNORECASE)
regex_lane_2 = re.compile(r' LN;', flags=re.IGNORECASE)

regex_drive_1 = re.compile(r' DR ', flags=re.IGNORECASE)
regex_drive_2 = re.compile(r' DR;', flags=re.IGNORECASE)

regex_close_1 = re.compile(r' CL ', flags=re.IGNORECASE)
regex_close_2 = re.compile(r' CL;', flags=re.IGNORECASE)

regex_place_1 = re.compile(r' PL ', flags=re.IGNORECASE)
regex_place_2 = re.compile(r' PL;', flags=re.IGNORECASE)

regex_ctrl_1 = re.compile(r' CTRL ', flags=re.IGNORECASE)
regex_ctrl_2 = re.compile(r' CTRL;', flags=re.IGNORECASE)

regex_cres_1 = re.compile(r' CRES ', flags=re.IGNORECASE)
regex_cres_2 = re.compile(r' CRES;', flags=re.IGNORECASE)

regex_terr_1 = re.compile(r' TER ', flags=re.IGNORECASE)
regex_terr_2 = re.compile(r' TER;', flags=re.IGNORECASE)

regex_ave_1 = re.compile(r' AVE ', flags=re.IGNORECASE)
regex_ave_2 = re.compile(r' AVE;', flags=re.IGNORECASE)

regex_nth_1 = re.compile(r' NTH ', flags=re.IGNORECASE)
regex_nth_2 = re.compile(r' NTH;', flags=re.IGNORECASE)
regex_nth_3 = re.compile(r'NTH ', flags=re.IGNORECASE)

regex_sth_1 = re.compile(r' STH ', flags=re.IGNORECASE)
regex_sth_2 = re.compile(r' STH;', flags=re.IGNORECASE)

regex_pk_1 = re.compile(r' PK ', flags=re.IGNORECASE)
regex_pk_2 = re.compile(r' PK;', flags=re.IGNORECASE)

regex_kg_1 = re.compile(r' KG ', flags=re.IGNORECASE)
regex_kg_2 = re.compile(r'KG ', flags=re.IGNORECASE)

regex_tg = re.compile(r'TG ', flags=re.IGNORECASE)

regex_upp = re.compile(r'UPP ', flags=re.IGNORECASE)
regex_lor = re.compile(r'LOR ', flags=re.IGNORECASE)


regex_jln = re.compile(r'JLN ', flags=re.IGNORECASE)

regex_hts = re.compile(r' HTS', flags=re.IGNORECASE)
regex_gdns = re.compile(r' GDNS', flags=re.IGNORECASE)
regex_mkt = re.compile(r' MKT ', flags=re.IGNORECASE)

regex_bt_merah = re.compile(r'BT MERAH', flags=re.IGNORECASE)
regex_bt_batok = re.compile(r'BT BATOK', flags=re.IGNORECASE)
regex_bt_panjang = re.compile(r'BT PANJANG', flags=re.IGNORECASE)
regex_bt_purmei = re.compile(r'BT PURMEI', flags=re.IGNORECASE)
regex_cwealth = re.compile(r"C'WEALTH", flags=re.IGNORECASE)

regex_clean = re.compile(r';', flags=re.IGNORECASE)


In [0]:
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_street_1,' STREET ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_street_2,' STREET')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_road_1,' ROAD ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_road_2,' ROAD')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_lane_1,' LANE ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_lane_2,' LANE')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_drive_1,' DRIVE ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_drive_2,' DRIVE')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_close_1,' CLOSE ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_close_2,' CLOSE')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_place_1,' PLACE ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_place_2,' PLACE')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_ctrl_1,' CENTRAL ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_ctrl_2,' CENTRAL')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_cres_1,' CRESCENT ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_cres_2,' CRESCENT')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_terr_1,' TERRACE ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_terr_2,' TERRACE')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_ave_1,' AVENUE ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_ave_2,' AVENUE')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_nth_1,' NORTH ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_nth_2,' NORTH')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_nth_3,'NORTH ')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_sth_1,' SOUTH ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_sth_2,' SOUTH')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_pk_1,' PARK ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_pk_2,' PARK')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_kg_1,' KAMPONG ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_kg_2,'KAMPONG ')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_tg,'TANJONG ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_upp,'UPPER ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_lor,'LORONG ')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_jln,'JALAN ')



street_names['updated_street'] = street_names['updated_street'].str.replace(regex_hts,' HEIGHTS')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_gdns,' GARDENS')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_mkt,' MARKET ')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_bt_merah,'BUKIT MERAH')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_bt_batok,'BUKIT BATOK')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_bt_panjang,'BUKIT PANJANG')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_bt_purmei,'BUKIT PURMEI')
street_names['updated_street'] = street_names['updated_street'].str.replace(regex_cwealth,'COMMONWEALTH')

street_names['updated_street'] = street_names['updated_street'].str.replace(regex_clean,'')

In [69]:
raw_address = list(street_names['street_name'])
cleaned_address = list(street_names['updated_street']) 

street_dict = {'raw_address':list(street_names['street_name']),
               'cleaned_address':list(street_names['updated_street'])
               }

street_addresses = pd.DataFrame(street_dict)
street_addresses.info()
street_addresses.head()
street_addresses.to_csv('streets_cleaned.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559 entries, 0 to 558
Data columns (total 2 columns):
raw_address        559 non-null object
cleaned_address    559 non-null object
dtypes: object(2)
memory usage: 8.8+ KB


,raw_address,cleaned_address
0,ANG MO KIO AVE 1,ANG MO KIO AVENUE 1
1,ANG MO KIO AVE 3,ANG MO KIO AVENUE 3
2,ANG MO KIO AVE 4,ANG MO KIO AVENUE 4
3,ANG MO KIO AVE 10,ANG MO KIO AVENUE 10
4,ANG MO KIO AVE 5,ANG MO KIO AVENUE 5


In [71]:
df = df.merge(street_addresses,'left',left_on='street_name',right_on='raw_address')
df.head()
df.info()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,resale_price,storey_range,street_name,town,resale_month,resale_year,lease_term_remaining,street,raw_address,cleaned_address
0,309,Improved,1R,31.0,1977,1990-01,9000.0,10-12,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990,0.0,ANG MO KIO AVE 1,ANG MO KIO AVE 1,ANG MO KIO AVENUE 1
1,309,Improved,1R,31.0,1977,1990-01,6000.0,04-06,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990,0.0,ANG MO KIO AVE 1,ANG MO KIO AVE 1,ANG MO KIO AVENUE 1
2,309,Improved,1R,31.0,1977,1990-01,8000.0,10-12,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990,0.0,ANG MO KIO AVE 1,ANG MO KIO AVE 1,ANG MO KIO AVENUE 1
3,309,Improved,1R,31.0,1977,1990-01,6000.0,07-09,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990,0.0,ANG MO KIO AVE 1,ANG MO KIO AVE 1,ANG MO KIO AVENUE 1
4,216,New Generation,3R,73.0,1976,1990-01,47200.0,04-06,ANG MO KIO AVE 1,ANG MO KIO,1990-01-01,1990,0.0,ANG MO KIO AVE 1,ANG MO KIO AVE 1,ANG MO KIO AVENUE 1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 804361 entries, 0 to 804360
Data columns (total 16 columns):
block                   804361 non-null object
flat_model              804361 non-null object
flat_type               804361 non-null object
floor_area_sqm          804361 non-null float64
lease_commence_date     804361 non-null int64
month                   804361 non-null object
resale_price            804361 non-null float64
storey_range            804361 non-null object
street_name             804361 non-null object
town                    804361 non-null object
resale_month            804361 non-null datetime64[ns]
resale_year             804361 non-null int64
lease_term_remaining    804361 non-null float64
street                  804361 non-null object
raw_address             804361 non-null object
cleaned_address         804361 non-null object
dtypes: datetime64[ns](1), float64(3), int64(2), object(10)
memory usage: 104.3+ MB


In [0]:
df['full_address'] = df['block'] + ', ' + df['cleaned_address']+ ', ' + df['town'] + ', SINGAPORE'

In [0]:
df_town_address = df[['full_address']]

In [88]:
df_town_address.drop_duplicates(inplace=True)
df_town_address.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9073 entries, 0 to 804306
Data columns (total 1 columns):
full_address    9073 non-null object
dtypes: object(1)
memory usage: 141.8+ KB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [89]:
df_town_address.reset_index(drop=True)

,full_address
0,"309, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE"
1,"216, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE"
2,"211, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE"
3,"202, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE"
4,"235, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE"
5,"232, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE"
6,"308, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE"
7,"220, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE"
8,"219, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE"
9,"247, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE"


In [80]:
df_lat_long = pd.read_csv('cloned-repo/df_geocode_list.csv')
df_lat_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8739 entries, 0 to 8738
Data columns (total 10 columns):
Unnamed: 0           8739 non-null int64
df_m.full_address    8739 non-null object
df.town              8739 non-null object
query                8739 non-null object
lat                  8739 non-null float64
lon                  8739 non-null float64
lat_min              8739 non-null float64
lat_max              8739 non-null float64
lon_min              8739 non-null float64
lon_max              8739 non-null float64
dtypes: float64(6), int64(1), object(3)
memory usage: 682.8+ KB


In [83]:
df_lat_long.columns=['unwanted','address','town','q','lat','lon','lat_min','lat_max','lon_min','lon_max']
df_lat_long.drop(columns=['unwanted','q','lat_min','lat_max','lon_min','lon_max'],inplace=True)
df_lat_long.head()


,address,town,lat,lon
0,"309, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE",ANG MO KIO,1.365305,103.838977
1,"216, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE",ANG MO KIO,1.365305,103.838977
2,"211, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE",ANG MO KIO,1.369204,103.843949
3,"202, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE",ANG MO KIO,1.368467,103.844154
4,"235, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE",ANG MO KIO,1.369204,103.843949


In [90]:
df_geo = df_town_address.merge(df_lat_long,'left',left_on='full_address',right_on='address')
df_geo.head()

,full_address,address,town,lat,lon
0,"309, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE","309, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE",ANG MO KIO,1.365305,103.838977
1,"216, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE","216, ANG MO KIO AVENUE 1, ANG MO KIO, SINGAPORE",ANG MO KIO,1.365305,103.838977
2,"211, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE","211, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE",ANG MO KIO,1.369204,103.843949
3,"202, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE","202, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE",ANG MO KIO,1.368467,103.844154
4,"235, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE","235, ANG MO KIO AVENUE 3, ANG MO KIO, SINGAPORE",ANG MO KIO,1.369204,103.843949


In [100]:
unmapped_addresses = list(df_geo[df_geo.isna().any(axis=1)]['full_address'])
unmapped_addresses

['259, ANG MO KIO AVENUE 2, ANG MO KIO, SINGAPORE',
 '33, TAMAN HO SWEE, BUKIT MERAH, SINGAPORE',
 '105, HENDERSON CRESCENT, BUKIT MERAH, SINGAPORE',
 '18, KAMPONG BAHRU HILL, BUKIT MERAH, SINGAPORE',
 '18, TAMAN HO SWEE, BUKIT MERAH, SINGAPORE',
 '6, JALAN BT HO SWEE, BUKIT MERAH, SINGAPORE',
 '101, HENDERSON CRESCENT, BUKIT MERAH, SINGAPORE',
 '103, HENDERSON CRESCENT, BUKIT MERAH, SINGAPORE',
 '7, KIM TIAN PLACE, BUKIT MERAH, SINGAPORE',
 '81, MACPHERSON LANE, GEYLANG, SINGAPORE',
 '1, JALAN PASAR BARU, GEYLANG, SINGAPORE',
 '114, HOUGANG AVENUE 1, HOUGANG, SINGAPORE',
 '110, HOUGANG AVENUE 1, HOUGANG, SINGAPORE',
 '142, LORONG AH SOO, HOUGANG, SINGAPORE',
 '135, LORONG AH SOO, HOUGANG, SINGAPORE',
 '137, LORONG AH SOO, HOUGANG, SINGAPORE',
 '154, HOUGANG STREET 11, HOUGANG, SINGAPORE',
 '138, LORONG AH SOO, HOUGANG, SINGAPORE',
 '140, LORONG AH SOO, HOUGANG, SINGAPORE',
 '234, HOUGANG AVENUE 1, HOUGANG, SINGAPORE',
 '231, HOUGANG STREET 21, HOUGANG, SINGAPORE',
 '133, LORONG AH SOO

In [102]:
geolocator = Nominatim(user_agent="sing_analysis")
geo_matched_details = []
unmatched_addresses = []
count_total = 0
count_unmatched = 0

for address in unmapped_addresses:
  count_total +=1
  try:
    location = geolocator.geocode(address)
    geo_matched_details.append(location.raw)
  except:
    unmatched_addresses.append(address)
    count_unmatched +=1
    print('{} unmatched out of {}'.format(count_unmatched, count_total))
  

  time.sleep(0.5)

1 unmatched out of 1
2 unmatched out of 2
3 unmatched out of 3
4 unmatched out of 4
5 unmatched out of 5
6 unmatched out of 6
7 unmatched out of 7
8 unmatched out of 8
9 unmatched out of 9
10 unmatched out of 10
11 unmatched out of 11
12 unmatched out of 26
13 unmatched out of 27
14 unmatched out of 28
15 unmatched out of 66
16 unmatched out of 67
17 unmatched out of 68
18 unmatched out of 69
19 unmatched out of 70
20 unmatched out of 71
21 unmatched out of 72
22 unmatched out of 77
23 unmatched out of 78
24 unmatched out of 96
25 unmatched out of 97
26 unmatched out of 98
27 unmatched out of 99
28 unmatched out of 100
29 unmatched out of 101
30 unmatched out of 102
31 unmatched out of 120
32 unmatched out of 121
33 unmatched out of 122
34 unmatched out of 123
35 unmatched out of 124
36 unmatched out of 125
37 unmatched out of 128
38 unmatched out of 141
39 unmatched out of 142
40 unmatched out of 144
41 unmatched out of 153
42 unmatched out of 159
43 unmatched out of 169
44 unmatched 

In [0]:
new_match_df = pd.DataFrame(geo_matched_details)
new_match_df.to_csv('new_geo.csv')

In [0]:
frame_by_town = [ frame for town, frame in df_town_address.groupby('town') ]

In [0]:
count = 0
for i in range(len(frame_by_town)):
  for address in frame_by_town[0]['full_address']:
    count += 1
  print(count)

365
730
1095
1460
1825
2190
2555
2920
3285
3650
4015
4380
4745
5110
5475
5840
6205
6570
6935
7300
7665
8030
8395
8760
9125
9490
9855


In [0]:
df.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town,full_address
0,309,Improved,1R,31.0,1977,1990-01-01,NaN,9000.0,10-12,ANG MO KIO AVE 1,ANG MO KIO,"309, ANG MO KIO AVE 1, ANG MO KIO, Singapore"
1,309,Improved,1R,31.0,1977,1990-01-01,NaN,6000.0,04-06,ANG MO KIO AVE 1,ANG MO KIO,"309, ANG MO KIO AVE 1, ANG MO KIO, Singapore"
2,309,Improved,1R,31.0,1977,1990-01-01,NaN,8000.0,10-12,ANG MO KIO AVE 1,ANG MO KIO,"309, ANG MO KIO AVE 1, ANG MO KIO, Singapore"
3,309,Improved,1R,31.0,1977,1990-01-01,NaN,6000.0,07-09,ANG MO KIO AVE 1,ANG MO KIO,"309, ANG MO KIO AVE 1, ANG MO KIO, Singapore"
4,216,New Generation,3R,73.0,1976,1990-01-01,NaN,47200.0,04-06,ANG MO KIO AVE 1,ANG MO KIO,"216, ANG MO KIO AVE 1, ANG MO KIO, Singapore"


In [0]:
df_addresses = df.full_address.unique()

In [0]:
len(df_addresses)

9073

In [0]:
geolocator = Nominatim(user_agent="sing_analysis")

In [0]:
df_addresses[0]

'309, ANG MO KIO AVE 1, ANG MO KIO, Singapore'

In [0]:
location = geolocator.geocode(df_addresses[0])

In [0]:
type(location)

geopy.location.Location

In [0]:
print(location.address, location.latitude, location.longitude)

Ang Mo Kio Avenue 1, Ang Mo Kio Neighbourhood Marker Sign, Ang Mo Kio, Singapore, Central, 560218, Singapore 1.3642639 103.8430657


In [0]:
print(location.raw)

{'place_id': 186984920, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 536925839, 'boundingbox': ['1.3642639', '1.3642769', '103.8429466', '103.8430657'], 'lat': '1.3642639', 'lon': '103.8430657', 'display_name': 'Ang Mo Kio Avenue 1, Ang Mo Kio Neighbourhood Marker Sign, Ang Mo Kio, Singapore, Central, 560218, Singapore', 'class': 'highway', 'type': 'primary', 'importance': 1.01}


In [0]:
geo_matched_details = []
unmatched_addresses = []
count_total = 0
count_unmatched = 0

for address in frame_by_town[0]['full_address']:
  count_total +=1
  try:
    location = geolocator.geocode(address)
    geo_details.append(location.raw)
  except:
    unmatched_addresses.append(address)
    count_unmatched +=1
    print('{} unmatched out of {}'.format(count_unmatched, count_total))
  

  time.sleep(2.0)

TypeError: ignored

In [0]:
len(geo_details)

361

In [0]:
df_geo = pd.DataFrame(geo_details)
df_geo.to_csv('geo_details.csv')


,boundingbox,class,display_name,icon,importance,lat,licence,lon,osm_id,osm_type,place_id,type
0,"[1.3692993, 1.3693734, 103.8434198, 103.8439545]",highway,"Ang Mo Kio Avenue 3, Ang Mo Kio Neighbourhood ...",NaN,1.010,1.3693734,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.8434198,634125474,way,241637748,primary
1,"[1.3692993, 1.3693734, 103.8434198, 103.8439545]",highway,"Ang Mo Kio Avenue 3, Ang Mo Kio Neighbourhood ...",NaN,1.010,1.3693734,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.8434198,634125474,way,241637748,primary
2,"[1.3692993, 1.3693734, 103.8434198, 103.8439545]",highway,"Ang Mo Kio Avenue 3, Ang Mo Kio Neighbourhood ...",NaN,1.010,1.3693734,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.8434198,634125474,way,241637748,primary
3,"[1.3712338, 1.3718964, 103.838697, 103.8393769]",building,"104A, Ang Mo Kio Street 11, Ang Mo Kio Neighbo...",NaN,1.021,1.3715979,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.839182125638,170062539,way,116708707,yes
4,"[1.3713185, 1.3716266, 103.8379929, 103.8389244]",building,"104B, Ang Mo Kio Street 11, Ang Mo Kio Neighbo...",NaN,1.021,1.37146775,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.838461534141,102413778,way,97765977,yes
5,"[1.3694537, 1.3699069, 103.8370094, 103.8372834]",highway,"Ang Mo Kio Avenue 4, Ang Mo Kio Neighbourhood ...",NaN,1.010,1.3695254,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.8372385,632096896,way,241071001,secondary
6,"[1.3694537, 1.3699069, 103.8370094, 103.8372834]",highway,"Ang Mo Kio Avenue 4, Ang Mo Kio Neighbourhood ...",NaN,1.010,1.3695254,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.8372385,632096896,way,241071001,secondary
7,"[1.3694537, 1.3699069, 103.8370094, 103.8372834]",highway,"Ang Mo Kio Avenue 4, Ang Mo Kio Neighbourhood ...",NaN,1.010,1.3695254,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.8372385,632096896,way,241071001,secondary
8,"[1.3694537, 1.3699069, 103.8370094, 103.8372834]",highway,"Ang Mo Kio Avenue 4, Ang Mo Kio Neighbourhood ...",NaN,1.010,1.3695254,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.8372385,632096896,way,241071001,secondary
9,"[1.3694537, 1.3699069, 103.8370094, 103.8372834]",highway,"Ang Mo Kio Avenue 4, Ang Mo Kio Neighbourhood ...",NaN,1.010,1.3695254,"Data © OpenStreetMap contributors, ODbL 1.0. h...",103.8372385,632096896,way,241071001,secondary
